# dask-sql
### A SQL Query Layer for Dask

## Introduction

`dask-sql` adds a SQL query layer on top of the Dask distributed Python library, which allows you to query your big and small data with SQL and still use the great power of the Dask ecosystem.
It helps you combine the best of both worlds.
See the [documentation](https://dask-sql.readthedocs.io/) for more information.

## Starting dask-sql

There are two possibilities how you can send your SQL queries to `dask-sql`:
* you use a Python notebook/script, such as the one you have currently opened
* you run the [dask-sql Server](https://dask-sql.readthedocs.io/en/latest/pages/server.html) as a standalone application and connect to it via e.g. your BI tool

We will stick with the first possibility in this notebook, but all SQL commands shown here can also be run via the SQL server.

Before we start, we need do import `dask-sql` and create a `Context`, which collects all the information on the currently registered data tables.
We will also create a small local Dask cluster (this step is not needed, but gives us a bit more debugging options).
If you have a large computation cluster, you can connect to it in this step (have a look [here](https://docs.dask.org/en/latest/setup.html)).

In [ ]:
from dask_sql import Context
from dask.distributed import Client

client = Client()
c = Context()

In [ ]:
client

You are now ready to query with SQL!

In [ ]:
c.sql("""
    SELECT 42 AS "the answer"
""", return_futures=False)

## Data Input

### 1. From a Dask Dataframe via Python

In [ ]:
import dask.dataframe as dd

df = dd.read_csv("./iris.csv")

In [ ]:
c.create_table("iris", df)

### 2. From an external data source via SQL

In [ ]:
c.sql("""
    CREATE OR REPLACE TABLE iris
    WITH (
        location = 'file://./iris.csv',
        format = 'csv',
        persist = True
    )
""")

### 3. From more "advanced" sources

* s3, azure, dbfs, gs, hdfs, ...
* hive (experimental)
* already loaded data persisted in your Dask cluster

More [information](https://dask-sql.readthedocs.io/en/latest/pages/data_input.html)

### 4. As materialized Queries

In [ ]:
c.sql("""
    CREATE OR REPLACE TABLE second_iris -- or CREATE VIEW 
    AS SELECT * FROM iris
""")

## Data Query

You can call "normal" SQL `SELECT` statements in `dask-sql`, with all typical components from the standard SQL language.
More information in the [SQL reference](https://dask-sql.readthedocs.io/en/latest/pages/sql.html).
`dask-sql` roughly follows the prestoSQL conventions (e.g. quoting).

<div class="alert alert-info">
    
#### Note
    
Not all SQL operators are implemented in `dask-sql` already.
    
</div>

In [ ]:
c.sql("""
    SELECT * 
    FROM iris
""")

In [ ]:
c.sql("""
    SELECT 
        sepal_length + sepal_width AS "sum", 
        SIN(petal_length) AS "sin"
    FROM iris
    LIMIT 10
""").compute()

In [ ]:
c.sql("""
    SELECT * 
    FROM iris
    LIMIT 10
""", return_futures=False)

In [ ]:
c.sql("""
    SELECT 
        species,
        AVG(sepal_length) AS sepal_length, 
        AVG(sepal_width) AS sepal_width
    FROM iris
    GROUP BY species
    LIMIT 10
""").compute()

In [ ]:
c.sql("""
    SELECT 
        species,
        AVG(sepal_length) AS sepal_length, 
        AVG(sepal_width) AS sepal_width
    FROM iris
    GROUP BY species
""").visualize()

In [ ]:
c.sql("""
    WITH maximal_values AS (
        SELECT 
            species, 
            MAX(sepal_length) AS sepal_length
        FROM iris
        GROUP BY species
    )
    SELECT 
        lhs.*
    FROM iris AS lhs 
    JOIN maximal_values AS rhs
    ON lhs.species = rhs.species 
        AND lhs.sepal_length = rhs.sepal_length
""").compute()

In [ ]:
print(c.explain("""
    WITH maximal_values AS (
        SELECT 
            species, 
            MAX(sepal_length) AS sepal_length
        FROM iris
        GROUP BY species
    )
    SELECT 
        lhs.*
    FROM iris AS lhs 
    JOIN maximal_values AS rhs
    ON lhs.species = rhs.species 
        AND lhs.sepal_length = rhs.sepal_length
"""))

## Metadata Information

In [ ]:
c.sql("""
    SHOW TABLES FROM "schema"
""").compute()

In [ ]:
c.sql("""
    SHOW COLUMNS FROM "iris"
""").compute()

## Custom Functions

In [ ]:
import numpy as np

def my_f(x, y):
    return x + y

# As SQL is a typed language, we need to specify all types 
c.functions.clear()
c.function_list.clear()
c.register_function(my_f, "MY_F", parameters=[("x", np.float64), ("y", np.float64)], return_type=np.float64)

In [ ]:
c.sql("""
    SELECT 
        sepal_length, sepal_width, MY_F(sepal_length, sepal_width) AS F
    FROM iris
    LIMIT 10
""").compute()

## Machine Learning

In [ ]:
c.sql("""
    CREATE OR REPLACE TABLE training_data AS (
        SELECT * FROM iris
        LIMIT 100
    ) 
""")

c.sql("""
    CREATE OR REPLACE MODEL my_model WITH (
        model_class = 'dask_ml.cluster.KMeans',
        n_clusters = 4
    ) AS (
        SELECT sepal_length, sepal_width, petal_length, petal_width FROM training_data
    )

""")

In [ ]:
c.sql("""
    SELECT
        *
    FROM PREDICT(MODEL my_model,
        SELECT * FROM iris
    )
""").compute()

In [ ]:
c.sql("""
    CREATE OR REPLACE TABLE results AS
    SELECT
        *
    FROM PREDICT(MODEL my_model,
        SELECT * FROM iris
    )
""")

In [ ]:
c.sql("""
    SELECT
        target,
        species,
        COUNT(*)
    FROM
        results
    GROUP BY target, species
""").compute()

In [ ]:
t = c.sql("""
    SELECT
        target,
        species,
        COUNT(*) AS "number"
    FROM
        results
    GROUP BY target, species
""").compute() 
t.set_index(["target", "species"]).unstack("species").number.plot.bar()